In [2]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

In [3]:
# 훈련 데이터 로드
data = pd.read_csv('merged_data.csv', encoding='utf-8')

In [4]:
# 필요한 열 선택
selected_features = ['stn_id', 'borrowed_hour', 'borrowed_day', 'is_holiday', 'borrowed_num_nearby', '강수량(mm)', 'wind_chill', 'stn_gu', 'nearby_id', 'borrowed_date', 'borrowed_num']
data = data[selected_features]

In [5]:
# 범주형 데이터를 숫자로 변환 (Label Encoding)
categorical_features = ['stn_id', 'stn_gu', 'nearby_id']
for feature in categorical_features:
    data[feature] = pd.factorize(data[feature])[0]

In [6]:
# 훈련-테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(data.drop('borrowed_num', axis=1), data['borrowed_num'], test_size=0.2, random_state=42)


In [7]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [8]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data, train_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3, verbose=100),
    ])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.290146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1136
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 10
[LightGBM] [Info] Start training from score 1.483498
Training until validation scores don't improve for 3 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 1.97365	valid_0's rmse: 1.98518


In [9]:
# 훈련 데이터에서의 평가
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
print(f'Training MSE: {mse_train}')
print(f'Training MAE: {mae_train}')
print(f'Training RMSE: {rmse_train}')

Training MSE: 3.8952857840571915
Training MAE: 1.1453070864914188
Training RMSE: 1.9736478368891426


In [10]:
# 검증 데이터에서의 평가
y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
print(f'Test MSE: {mse_test}')
print(f'Test MAE: {mae_test}')
print(f'Test RMSE: {rmse_test}')

Test MSE: 3.940949911399397
Test MAE: 1.1485539408033907
Test RMSE: 1.985182588932161


In [40]:
# 새로운 데이터 예측
new_data = pd.read_csv('data/new_data.csv', encoding='utf-8')  # 'new_data.csv'는 새로운 데이터 파일명으로 대체해야 합니다.


In [41]:
new_data.head()

,stn_id,borrowed_hour,borrowed_day,is_holiday,borrowed_num_nearby,강수량(mm),wind_chill,stn_gu,nearby_id,borrowed_date,borrowed_num
0,ST-1611,14,6,1,2,0.0,-3,노원구,ST-5306,20231201,18
1,ST-2915,15,6,1,1,0.0,-1,노원구,ST-5319,20231201,13
2,ST-10,12,6,1,3,0.0,1,마포구,ST-2167,20231201,1
3,ST-10,15,6,1,2,0.0,-4,마포구,ST-2167,20231201,2
4,ST-10,20,6,1,0,0.0,0,마포구,ST-2167,20231201,2


In [45]:
# 훈련 데이터에서 사용된 특성 목록
used_features = X_train.columns.tolist()

In [46]:
new_data_selected = new_data[selected_features]

In [47]:
for feature in categorical_features:
    new_data_selected[feature] = pd.factorize(new_data_selected[feature])[0]  # 범주형 데이터를 숫자로 변환

In [49]:
y_pred_new = model.predict(new_data_selected, num_iteration=model.best_iteration)
print(f'Predictions for new data: {y_pred_new}')

LightGBMError: The number of features in data (11) is not the same as it was in training data (10).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.